In [72]:
#Import some modules
%matplotlib inline
import numpy as np
import pandas as pd
import utils as ut
import datetime as dt



#Load in the data
df1 = pd.read_csv('LoanStats3a.csv', skiprows = 1, 
                  parse_dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'],
                 skipfooter = 2)
df2 = pd.read_csv('LoanStats3b.csv', skiprows = 1, 
                  parse_dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'],
                 skipfooter = 2, infer_datetime_format=True)
df3 = pd.read_csv('LoanStats3c.csv', skiprows = 1, 
                  parse_dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'],
                 skipfooter = 2, infer_datetime_format=True)
df4 = pd.read_csv('LoanStats3d.csv', skiprows = 1, 
                  parse_dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'],
                 skipfooter = 2, infer_datetime_format=True)

loans = pd.concat([df1, df2, df3, df4], ignore_index = True)

del df1
del df2
del df3
del df4

In [73]:
#Change 'does not meet...' fields 
loans.replace(to_replace = 'Does not meet the credit policy. Status:Fully Paid', value = 'Fully Paid', inplace = True)
loans.replace(to_replace = 'Does not meet the credit policy. Status:Charged Off', value = 'Charged Off', inplace = True)

#Assign default to charged off
loans.replace(to_replace = 'Default', value = 'Charged Off', inplace = True)

In [74]:
#Filter out everything but 'fully paid' and 'charged off'
loans = loans[(loans.loan_status == 'Fully Paid') | (loans.loan_status == 'Charged Off')]

In [77]:
#annual_inc is null in four places where a whole bunch of other values are null
#drop the four null entries for annual_inc
loans.dropna(subset = ['annual_inc'], inplace = True)

In [78]:
#delinq_2yrs, inq_last_6mths, open_acc, pub_rec, total_acc, acc_now_delinq
#all share 25 rows where all are null, drop those rows
loans.dropna(subset = ['delinq_2yrs', 'inq_last_6mths', 
                      'open_acc', 'pub_rec', 'acc_now_delinq'], inplace = True)

In [79]:
#116 null values in collections_12_mths_ex_med, set those to the median value
loans.collections_12_mths_ex_med.fillna(loans.collections_12_mths_ex_med.median(), inplace = True)

In [80]:
#Clean home_ownership
#There is one loan with home_ownwership == 'ANY', add that to 'OTHER'
loans.home_ownership.replace(to_replace = 'ANY', value = 'OTHER', inplace = True)

In [81]:
#Clean emp_length
#Create a dictionary to change the year strings into numbers
year_dict = {'emp_length':{'10+ years':10, '< 1 year':0, '3 years':3, '9 years':9, '4 years':4, '5 years':5,
       '1 year':1, '6 years':6, '2 years':2, '7 years':7, '8 years':8, 'n/a':-1}}
#Replace the year strings with numbers
loans.replace(year_dict, inplace = True)

In [82]:
#Clean revol_util
#nans are rare, there are no zeros, pretty sure nan is either a zero or decline to report
#replace nans with 0%
loans.revol_util.fillna('0%', inplace = True)

#Create a dictionary to convert revol_util strings to floats
revol_ut_dict = {revol_ut: float(revol_ut[0:-1]) for revol_ut in loans.revol_util.unique()}

#Replace string revol_util with floats
loans.replace(to_replace = {'revol_util': revol_ut_dict}, inplace = True)

In [83]:
#Process the dates
base_time = dt.datetime.now()
#Parse the dates into datetime objects
cr_line = ut.parse_date_series(loans.earliest_cr_line)
pl_date = ut.parse_date_series(loans.last_credit_pull_d)

In [84]:
#Fill (few) null times with the current time
cr_line.fillna(value = base_time, inplace = True)
pl_date.fillna(value = base_time, inplace = True)

In [85]:
#Get the months since first credit and last credit pull
cr_line_months = ut.months_since(base_time, cr_line)
pl_date_months = ut.months_since(base_time, pl_date)

In [86]:
#Change null values into the means
cr_line_months.replace(to_replace=0, value=cr_line_months.mean(), inplace=True)
pl_date_months.replace(to_replace=0, value=pl_date_months.mean(), inplace=True)

In [87]:
#Add the new columns to the data
loans['erlst_cred'] = cr_line_months
loans['last_cred_pl'] = pl_date_months

del cr_line_months
del pl_date_months

In [88]:
#Read in IRS zipcode data
zip_data = pd.read_pickle('anon_avg_irs_data.pkl')

In [89]:
#Create a dictionary to convert anonomized string zipcodes into integers
code_dict = {code: int(code[0:3]) for code in loans.zip_code.unique()}
#Replace string zipcodes with integer zipcodes
loans.replace(to_replace = {'zip_code': code_dict}, inplace = True)

In [90]:
#Drop unnecessary columns from zipcode data
zip_data.drop(['adj_gross_inc', 'amt_SS', 'amt_edu', 'amt_itemized',
       'amt_mort_intr', 'amt_st_loans', 'amt_unemp', 'n_SS', 'n_edu',
       'n_farm', 'n_itemized', 'n_mort_intr', 'n_returns', 'n_st_loans',
       'n_unemp'], axis = 1, inplace=True)
#Rename the zipcd column so that we can do a join
zip_data.rename(columns = {'zipcd':'zip_code'}, inplace = True)

In [91]:
#Join zipcode data to loan data on zip_code
loans = loans.join(zip_data, on = 'zip_code', rsuffix = 'r')
loans.drop('zip_coder', axis = 1, inplace = True)
del zip_data

In [101]:
#Some zip code data mismatch, fill missing indices with medians
zip_fill_dict = {'avg_inc': loans.avg_inc.median(), 'avg_SS': loans.avg_SS.median(), 
                 'avg_itemized': loans.avg_itemized.median(), 'avg_unemp': loans.avg_unemp.median(),
                 'avg_mort_intr': loans.avg_mort_intr.median(), 'avg_edu': loans.avg_edu.median(),
                 'avg_st_loans': loans.avg_st_loans.median(), 'prop_SS': loans.prop_SS.median(),
                 'prop_itemized': loans.prop_itemized.median(), 'prop_unemp': loans.prop_unemp.median(),
                 'prop_farm': loans.prop_farm.median(), 'prop_mort_intr': loans.prop_mort_intr.median(),
                 'prop_edu': loans.prop_edu.median(), 'prop_st_loans': loans.prop_st_loans.median()}
loans.fillna(value = zip_fill_dict, inplace = True)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,avg_mort_intr,avg_edu,avg_st_loans,prop_SS,prop_itemized,prop_unemp,prop_farm,prop_mort_intr,prop_edu,prop_st_loans
0,1077501,1296599,5000,5000,4975.000000,36 months,10.65%,162.87,B,B2,...,10.762491,1.924419,0.956840,0.096519,0.294366,0.072245,6.766248e-03,0.236607,0.019358,0.095439
1,1077430,1314167,2500,2500,2500.000000,60 months,15.27%,59.83,C,C4,...,6.943484,2.613861,1.033616,0.107855,0.288517,0.060976,2.474951e-03,0.207552,0.011425,0.066625
2,1077175,1313524,2400,2400,2400.000000,36 months,15.96%,84.33,C,C5,...,10.577200,2.579682,1.107968,0.063228,0.261276,0.090446,1.632616e-04,0.195071,0.016173,0.096599
3,1076863,1277178,10000,10000,10000.000000,36 months,13.49%,339.31,C,C1,...,13.314386,2.126733,0.984944,0.077820,0.348774,0.108607,2.547390e-04,0.270798,0.013350,0.060813
5,1075269,1311441,5000,5000,5000.000000,36 months,7.90%,156.46,A,A4,...,11.948597,1.847579,0.978844,0.131628,0.399856,0.059974,9.853996e-04,0.326263,0.018635,0.094218
7,1072053,1288686,3000,3000,3000.000000,36 months,18.64%,109.43,E,E1,...,16.224948,2.187500,1.033117,0.055432,0.266141,0.097907,8.316176e-05,0.152030,0.010104,0.065829
8,1071795,1306957,5600,5600,5600.000000,60 months,21.28%,152.39,F,F2,...,10.958611,1.969444,0.933468,0.104369,0.302070,0.118394,5.216978e-04,0.242813,0.011351,0.071174
9,1071570,1306721,5375,5375,5350.000000,60 months,12.69%,121.45,B,B5,...,9.030859,2.250278,1.024369,0.081780,0.373263,0.055700,1.862762e-02,0.308522,0.015844,0.079372
10,1070078,1305201,6500,6500,6500.000000,60 months,14.65%,153.45,C,C3,...,9.909097,1.830481,0.942112,0.146695,0.342892,0.082828,1.436057e-03,0.284663,0.015023,0.080841
11,1069908,1305008,12000,12000,12000.000000,36 months,12.69%,402.54,B,B5,...,15.780425,1.981829,0.994504,0.100362,0.425365,0.103193,3.538255e-04,0.330313,0.013464,0.060862


In [102]:
#Drop all columns not being used
drop_list = ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
            'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'verification_status', 'issue_d',
            'pymnt_plan', 'url', 'desc', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'initial_list_status',
            'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
            'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
            'next_pymnt_d', 'last_credit_pull_d', 'policy_code', 'application_type', 'annual_inc_joint',
            'dti_joint', 'verification_status_joint', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m',
            'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m',
            'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m','max_bal_bc', 'all_util',
            'inq_fi', 'total_fi_tl', 'inq_last_12m']
loans.drop(drop_list, axis = 1, inplace = True)

In [103]:
#Change categorical variables into indicator variables
#Change loan status to a number "charged off" is signal, so 1, "fully paid" is 0
loans.replace({'loan_status': {'Fully Paid':0, 'Charged Off':1}}, inplace = True)
#Change home_ownership variables (leaving NONE out)
loans['has_mortgage'] = pd.Series(data=loans.home_ownership == 'MORTGAGE', index=loans.index)
loans['has_rent'] = pd.Series(data=loans.home_ownership == 'RENT', index=loans.index)
loans['has_own'] = pd.Series(data=loans.home_ownership == 'OWN', index=loans.index)
loans['has_other'] = pd.Series(data=loans.home_ownership == 'OTHER', index=loans.index)
#Change purpose varaiables (leaving out 'educational)
loans['for_car'] = pd.Series(data=loans.purpose == 'car', index=loans.index)
loans['for_cc'] = pd.Series(data=loans.purpose == 'credit_card', index=loans.index)
loans['for_debt'] = pd.Series(data=loans.purpose == 'debt_consolidation', index=loans.index)
loans['for_home_imp'] = pd.Series(data=loans.purpose == 'home_improvement', index=loans.index)
loans['for_house'] = pd.Series(data=loans.purpose == 'house', index=loans.index)
loans['for_purchase'] = pd.Series(data=loans.purpose == 'major_purchase', index=loans.index)
loans['for_med'] = pd.Series(data=loans.purpose == 'medical', index=loans.index)
loans['for_move'] = pd.Series(data=loans.purpose == 'moving', index=loans.index)
loans['for_other'] = pd.Series(data=loans.purpose == 'other', index=loans.index)
loans['for_energy'] = pd.Series(data=loans.purpose == 'renewable_energy', index=loans.index)
loans['for_business'] = pd.Series(data=loans.purpose == 'small_business', index=loans.index)
loans['for_vacation'] = pd.Series(data=loans.purpose == 'vacation', index=loans.index)
loans['for_wedding'] = pd.Series(data=loans.purpose == 'wedding', index=loans.index)
#Drop purpose and home_ownership columns
loans.drop(['purpose', 'home_ownership'], axis = 1, inplace = True)

In [104]:
#Pull out the labels and features
labels = loans['loan_status']
features = loans.drop('loan_status', axis = 1)

In [105]:
#Create a data frame for imputing the features
drop_list = ['mths_since_last_delinq', 'mths_since_last_record', 'mths_since_last_major_derog',
             'total_credit_rv', 'tot_coll_amt']
features_impute = features.drop(drop_list, axis = 1)
#Get the features for imputation
since_delinq = features['mths_since_last_delinq']
since_record = features['mths_since_last_record']
since_derog = features['mths_since_last_major_derog']
total_cred = features['total_credit_rv']
total_coll = features['tot_coll_amt']

In [107]:
#Get the known and unknown components for the values and features
since_delinq_unknown = since_delinq[since_delinq.isnull()]
since_delinq_known = since_delinq[since_delinq.isnull() == False]
delinq_feats_unknown = features_impute[since_delinq.isnull()]
delinq_feats_known = features_impute[since_delinq.isnull() == False]

since_record_unknown = since_record[since_record.isnull()]
since_record_known = since_record[since_record.isnull() == False]
record_feats_unknown = features_impute[since_record.isnull()]
record_feats_known = features_impute[since_record.isnull() == False]

since_derog_unknown = since_derog[since_derog.isnull()]
since_derog_known = since_derog[since_derog.isnull() == False]
derog_feats_unknown = features_impute[since_derog.isnull()]
derog_feats_known = features_impute[since_derog.isnull() == False]

total_cred_unknown = total_cred[total_cred.isnull()]
total_cred_known = total_cred[total_cred.isnull() == False]
cred_feats_unknown = features_impute[total_cred.isnull()]
cred_feats_known = features_impute[total_cred.isnull() == False]

total_coll_unknown = total_cred[total_coll.isnull()]
total_coll_known = total_cred[total_coll.isnull() == False]
coll_feats_unknown = features_impute[total_coll.isnull()]
coll_feats_known = features_impute[total_coll.isnull() == False]

In [108]:
from sklearn.preprocessing import MinMaxScaler
test = delinq_feats_unknown.values
scaler = MinMaxScaler()
scale_test = scaler.fit_transform(test)
scale_test

array([[ 1.        ,  0.00237841,  0.69142286, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.09090909,  0.00306781,  0.02500625, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.00102858,  0.21805451, ...,  1.        ,
         0.        ,  0.        ],
       ..., 
       [ 1.        ,  0.00686844,  0.79269817, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.27272727,  0.00272311,  0.46461615, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.00536579,  0.31582896, ...,  0.        ,
         0.        ,  0.        ]])